<a href="https://colab.research.google.com/github/anilkumarpanda/fairness_cb/blob/development/Testing_and_Remediating_Bias_constrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/ml-for-high-risk-apps-book/Machine-Learning-for-High-Risk-Applications-Book/blob/main/code/Chapter-10/Testing_and_Remediating_Bias_constrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fairness Code Breakfast : Testing and Remediating Bias in an XGBoost Credit Decision Model.

<img src="https://raw.githubusercontent.com/anilkumarpanda/fairness_cb/development/img/adog.JPG" alt="Alt Text" height="400">

This AI generated dog image has nothing to do with Fairness, but I hope I have your attention now.



# Fairness in Machine Learning ?

1. Is it important to consider fairness while developing models ?

2. Can it be achieved via a model ?

### 1. Setting the environment

Download the data.

In [ ]:
!wget https://raw.githubusercontent.com/anilkumarpanda/fairness_cb/development/credit_line_increase.csv

In [ ]:
# Installing the libraries
%pip install h2o
%pip install shap
%pip install 'XGBoost==1.6'

## 2. Evaluating an XGBoost Model




###  Excercise : Train a Credit Decision Model

Run the code/cells till #3. Measure Fairness.
We train a ML model, which some nice tricks.

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sklearn
import shap
np.random.seed(42)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/content/credit_line_increase.csv')
data['SEX'] = np.where(data['SEX'] == 1, 'male', 'female')
race_map = {1: 'hispanic', 2: 'black', 3: 'white', 4: 'asian'}
data['RACE'] = data['RACE'].apply(lambda x: race_map[x])

In [ ]:
# Modify the data so there is a distributional difference between borrowers of different race/ethnicities.

new_limit_bal = data['LIMIT_BAL'] - 20000*np.random.randn(len(data))
new_limit_bal[new_limit_bal <= 10000] = 10000
data['LIMIT_BAL'] = np.where((data['RACE'] == 'hispanic') | (data['RACE'] == 'black'),
                             new_limit_bal,
                             data['LIMIT_BAL'])

for i in range(1, 7):
    delta = 1000*np.random.randn(len(data))
    new_pay = data[f'PAY_AMT{i}'] - delta
    new_pay[new_pay < 0] = 0

    new_bill = data[f'BILL_AMT{i}'] - delta
    new_bill[new_bill < 0] = 0

    data[f'PAY_AMT{i}'] = np.where((data['RACE'] == 'hispanic') | (data['RACE'] == 'black'),
                                   new_pay,
                                   data[f'PAY_AMT{i}'])
    data[f'BILL_AMT{i}'] = np.where((data['RACE'] == 'hispanic') | (data['RACE'] == 'black'),
                                    new_bill,
                                    data[f'BILL_AMT{i}'])


In [ ]:
# Split the data into train validation and test
seed = 12345
np.random.seed(seed)

split_train_test = 2/3

split = np.random.rand(len(data)) < split_train_test
train = data[split].copy()
test = data[~split].copy()

split_test_valid = 1/2

split = np.random.rand(len(test)) < split_test_valid
valid = test[split].copy()
test = test[~split].copy()

del data

print(f"Train/Validation/Test sizes: {len(train)}/{len(valid)}/{len(test)}")

In [ ]:
id_col = 'ID'
groups = ['SEX', 'RACE', 'EDUCATION', 'MARRIAGE', 'AGE']
target = 'DELINQ_NEXT'
features = [col for col in train.columns if col not in groups + [id_col, target]]

dtrain = xgb.DMatrix(train[features],
                     label=train[target])

dvalid = xgb.DMatrix(valid[features],
                     label=valid[target])

In [ ]:
corr = pd.DataFrame(train[features + [target]].corr(method='spearman')[target]).iloc[:-1]

def get_monotone_constraints(data, target, corr_threshold):
    """Calculate monotonic constraints.

    Using a cutoff on Spearman correlation between features and target, return a tuple ready to pass into XGBoost.


    Args:
        data (pd.DataFrame): A DataFrame containing the features in the order they appear to XGBoost, as well as the target variable.
        target (str): The name of the column with the target variable in 'data'.
        corr_threshold (float): The Spearman correlation threshold.

    Returns:
        tuple: A tuple with values in {-1, 0, 1}, where each element corresponds to a column in data (excluding the target itself). Ready to pass into xgb.train()

    """

    corr = pd.Series(data.corr(method='spearman')[target]).drop(target)
    monotone_constraints = tuple(np.where(corr < -corr_threshold,
                                          -1,
                                          np.where(corr > corr_threshold,
                                                   1,
                                                   0)))
    return monotone_constraints

correlation_cutoff = 0.1

monotone_constraints = get_monotone_constraints(train[features+[target]],
                                                target,
                                                correlation_cutoff)

In [ ]:
# Feed the model the global bias
base_score = train[target].mean()

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': 0.05,
    'subsample': 0.6,
    'colsample_bytree': 1.0,
    'max_depth': 5,
    'base_score': base_score,
    'monotone_constraints': dict(zip(features, monotone_constraints)),
    'seed': seed
}

# Train using early stopping on the validation dataset.
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

model_constrained = xgb.train(params,
                              dtrain,
                              num_boost_round=200,
                              evals=watchlist,
                              early_stopping_rounds=10,
                              verbose_eval=False)

train[f'p_{target}'] = model_constrained.predict(dtrain)
valid[f'p_{target}'] = model_constrained.predict(dvalid)
test[f'p_{target}'] = model_constrained.predict(xgb.DMatrix(test[features], label=test[target]))

In [ ]:
# Select the optimal probability cutoff by maximizing the F1 score on validation data.

def perf_metrics(y_true, y_score, pos=1, neg=0, res=0.01):
    """
    Calculates precision, recall, and f1 given outcomes and probabilities.

    Args:
        y_true: Array of binary outcomes
        y_score: Array of assigned probabilities.
        pos: Primary target value, default 1.
        neg: Secondary target value, default 0.
        res: Resolution by which to loop through cutoffs, default 0.01.

    Returns:
        Pandas dataframe of precision, recall, and f1 values.
    """

    eps = 1e-20 # for safe numerical operations

    # init p-r roc frame
    prauc_frame = pd.DataFrame(columns=['cutoff', 'recall', 'precision', 'f1'])

    # loop through cutoffs to create p-r roc frame
    for cutoff in np.arange(0, 1 + res, res):

        # binarize decision to create confusion matrix values
        decisions = np.where(y_score > cutoff , 1, 0)

        # calculate confusion matrix values
        tp = np.sum((decisions == pos) & (y_true == pos))
        fp = np.sum((decisions == pos) & (y_true == neg))
        tn = np.sum((decisions == neg) & (y_true == neg))
        fn = np.sum((decisions == neg) & (y_true == pos))

        # calculate precision, recall, and f1
        recall = (tp + eps)/((tp + fn) + eps)
        precision = (tp + eps)/((tp + fp) + eps)
        f1 = 2/((1/(recall + eps)) + (1/(precision + eps)))


        # add new values to frame
        prauc_frame = prauc_frame.append({'cutoff': cutoff,
                                          'recall': recall,
                                          'precision': precision,
                                          'f1': f1},
                                          ignore_index=True)

    return prauc_frame


model_metrics = perf_metrics(y_true=valid[target], y_score=model_constrained.predict(dvalid))


In [ ]:
model_metrics.loc[model_metrics['f1'].idxmax()]

In [ ]:
# Since the Disparate Impact Analysis(DIA) analysis will focus on model outcomes
# (rather than scores), choose a cutoff in probability space.

best_cut = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']
best_cut_original = best_cut

fig, ax = plt.subplots()
ax.plot(model_metrics['cutoff'], model_metrics['precision'], label='Precision',linestyle='--')
ax.plot(model_metrics['cutoff'], model_metrics['recall'], label='Recall',linestyle=':')
ax.plot(model_metrics['cutoff'], model_metrics['f1'], label='F1 Score',linestyle='-.')
ax.legend(loc=3)
ax.set_xlabel('Score Cutoff')


In [ ]:
def get_confusion_matrix(frame, y, yhat, by=None, level=None, cutoff=0.5):
    """
    Creates confusion matrix from pandas dataframe of y and yhat values, can be sliced by a variable and level.

    Args:
        frame: Pandas dataframe of actual (y) and predicted (yhat) values.
        y: Name of actual value column.
        yhat: Name of predicted value column.
        by: By variable to slice frame before creating confusion matrix, default None.
        level: Value of by variable to slice frame before creating confusion matrix, default None.
        cutoff: Cutoff threshold for confusion matrix, default 0.5.

    Returns:
        Confusion matrix as pandas dataframe.
    """

    # determine levels of target (y) variable
    # sort for consistency
    level_list = list(frame[y].unique())
    level_list.sort(reverse=True)

    # init confusion matrix
    cm_frame = pd.DataFrame(columns=['actual: ' +  str(i) for i in level_list],
                            index=['predicted: ' + str(i) for i in level_list])

    # don't destroy original data
    frame_ = frame.copy(deep=True)

    # convert numeric predictions to binary decisions using cutoff
    dname = 'd_' + str(y)
    frame_[dname] = np.where(frame_[yhat] > cutoff , 1, 0)

    # slice frame
    if (by is not None) & (level is not None):
        frame_ = frame_[frame[by] == level]

    # calculate size of each confusion matrix value
    for i, lev_i in enumerate(level_list):
        for j, lev_j in enumerate(level_list):
            cm_frame.iat[j, i] = frame_[(frame_[y] == lev_i) & (frame_[dname] == lev_j)].shape[0]

    return cm_frame

get_confusion_matrix(test, target, f'p_{target}', cutoff=best_cut)

## 3. Measure Fairness


The model you created above will be used provide loans. So it is important to create measure the models for fairness.

In the domain of fairness, there a lot of metrics that can be considered depending on the problem.

Some of them include :

Prevalence: '(tp + fn) / (tp + tn +fp + fn)', # How much default actually happens for this group

Accuracy: '(tp + tn) / (tp + tn + fp + fn)', # how often the model predicts default and non-default correctly for this group

True Positive Rate: 'tp / (tp + fn)',  # out of the people in the group *that did* default, how many the model predicted *correctly* would default

Precision: 'tp / (tp + fp)',  # out of the people in the group the model *predicted* would default, how many the model predicted *correctly* would default

Specificity: 'tn / (tn + fp)', # out of the people in the group *that did not* default, how many the model predicted *correctly* would not default

Negative Predicted Value: 'tn / (tn + fn)', # out of the people in the group the model *predicted* would not default, how many the model predicted *correctly* would not default

False Positive Rate: 'fp / (tn + fp)', # out of the people in the group *that did not* default, how many the model predicted *incorrectly* would default

False Discovery Rate: 'fp / (tp + fp)', # out of the people in the group the model *predicted* would default, how many the model predicted *incorrectly* would default

False Negative Rate: 'fn / (tp + fn)', # out of the people in the group *that did* default, how many the model predicted *incorrectly* would not default

False Omissions Rate: 'fn / (tn + fn)'  # out of the people in the group the model *predicted* would not default, how many the model predicted *incorrectly* would not default

Are you feeling overwhelmed ??

<img src="https://raw.githubusercontent.com/anilkumarpanda/fairness_cb/development/img/blue_cat.JPG" alt="Sad Bleu Cat" height="400">



<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTkPZ1nf-X_zq_stiiNqv7opkVZU7wEJsMcjAhJSrq2ug&s" alt="Andrew Ng" height="400">

The Fairness Decision Tree can be a good starting point. ![fairness decision tree](https://raw.githubusercontent.com/anilkumarpanda/fairness_cb/development/fairness_tree.png)


However for our case we will focus on AIR , SMD across groups.

AIR : Adverse Impact Ratio

SMD : Statistical Measures of Disparate Impact.



### Confusion Matrix Disparity Metrics

In [ ]:
metric_dict = {
'Prevalence': '(tp + fn) / (tp + tn +fp + fn)', # how much default actually happens for this group
'Accuracy': '(tp + tn) / (tp + tn + fp + fn)', # how often the model predicts default and non-default correctly for this group
'True Positive Rate': 'tp / (tp + fn)',  # out of the people in the group *that did* default, how many the model predicted *correctly* would default
'Precision': 'tp / (tp + fp)',  # out of the people in the group the model *predicted* would default, how many the model predicted *correctly* would default
'Specificity': 'tn / (tn + fp)', # out of the people in the group *that did not* default, how many the model predicted *correctly* would not default
'Negative Predicted Value': 'tn / (tn + fn)', # out of the people in the group the model *predicted* would not default, how many the model predicted *correctly* would not default

'False Positive Rate': 'fp / (tn + fp)', # out of the people in the group *that did not* default, how many the model predicted *incorrectly* would default
'False Discovery Rate': 'fp / (tp + fp)', # out of the people in the group the model *predicted* would default, how many the model predicted *incorrectly* would default

'False Negative Rate': 'fn / (tp + fn)', # out of the people in the group *that did* default, how many the model predicted *incorrectly* would not default
'False Omissions Rate': 'fn / (tn + fn)'  # out of the people in the group the model *predicted* would not default, how many the model predicted *incorrectly* would not default
}


def confusion_matrix_parser(expression):

    # tp | fp       cm_dict[level].iat[0, 0] | cm_dict[level].iat[0, 1]
    # -------  ==>  --------------------------------------------
    # fn | tn       cm_dict[level].iat[1, 0] | cm_dict[level].iat[1, 1]

    expression = expression.replace('tp', 'cm_dict[level].iat[0, 0]')\
                           .replace('fp', 'cm_dict[level].iat[0, 1]')\
                           .replace('fn', 'cm_dict[level].iat[1, 0]')\
                           .replace('tn', 'cm_dict[level].iat[1, 1]')

    return expression

In [ ]:
# initialize dict of confusion matrices and corresponding rows of dataframe
sex_confusion_mats = {'male': get_confusion_matrix(test, target,
                                                   f'p_{target}', by='SEX',
                                                   level='male', cutoff=best_cut),
                      'female': get_confusion_matrix(test, target,
                                                     f'p_{target}', by='SEX',
                                                     level='female', cutoff=best_cut)}

def confusion_matrix_metrics(cm_dict, metric_dict):
    levels = list(cm_dict.keys())

    metrics_frame = pd.DataFrame(index=levels) # frame for metrics

    for level in levels:
        for metric in metric_dict.keys():

            # parse metric expressions into executable pandas statements
            expression = confusion_matrix_parser(metric_dict[metric])

            # dynamically evaluate metrics to avoid code duplication
            metrics_frame.loc[level, metric] = eval(expression)

    return metrics_frame

sex_confusion_metrics = confusion_matrix_metrics(sex_confusion_mats, metric_dict)

In [ ]:
sex_confusion_metrics

### Excercise :
1. Calculate metrics across Race groups.
2. When do we know if the metrics are in red ?
3. Can you highlight what values are in red ?

In [ ]:
# @title Excerise : Confusion Matrix for Race Group

race_levels = list(race_map.values())
race_confusion_mats = {level: get_confusion_matrix(test, target, f'p_{target}', by='RACE',
                                                   level=level, cutoff=best_cut) for level in race_levels}
race_confusion_metrics = confusion_matrix_metrics(race_confusion_mats, metric_dict)
race_confusion_metrics

In [ ]:
# @title Confusion Matrix for Race Group : Highlight disparity

race_disparity_frame = race_confusion_metrics/race_confusion_metrics.loc['white', :]
race_disparity_frame.columns=[col + ' Disparity' for col in race_confusion_metrics.columns]

# small utility function to format pandas table output
def disparate_red(val, parity_threshold_low=0.8, parity_threshold_hi=1.25):
    color = 'grey' if (parity_threshold_low < val < parity_threshold_hi) else 'red'
    return 'color: %s' % color

race_disparity_frame.style.applymap(disparate_red)

In [ ]:
# @title Confusion Matrix for Gender Group : Highlight disparity

sex_disparity_frame = sex_confusion_metrics/sex_confusion_metrics.loc['male', :]
sex_disparity_frame.columns=[col + ' Disparity' for col in sex_confusion_metrics.columns]

# small utility function to format pandas table output
def disparate_red(val, parity_threshold_low=0.8, parity_threshold_hi=1.25):
    color = 'grey' if (parity_threshold_low < val < parity_threshold_hi) else 'red'
    return 'color: %s' % color

sex_disparity_frame.style.applymap(disparate_red)

### Fair Lending Disparity Analysis

In [ ]:
from scipy.stats import ttest_ind, chisquare, fisher_exact, chi2_contingency


# def air_statistical_signif(group_count, group_favorable, reference_count, reference_favorable):
#     # Perform a chi-square test
#     # (or Fisher's exact when cells in the contingency test have less than 30 individuals in them).

#     group_unfavorable = group_count - group_favorable
#     reference_unfavorable = reference_count - reference_favorable

#     contingency_table = np.array([[group_favorable, group_unfavorable],
#                                   [reference_favorable, reference_unfavorable]])

#     if np.min(contingency_table) < 30:
#         _, p = fisher_exact(contingency_table)
#     else:
#         _, p, _, _ = chi2_contingency(contingency_table)

#     return p

# def smd_statistical_signif(group_scores, reference_scores):
#     # Perform a one-sided t-test. An outcome of 1 is assumed to be favorable.

#     # We do not assume that the two scores have equal variance. Furthermore, we are testing
#     # against the alternative hypothesis that the group receives lower scores than the reference
#     # group.
#     _, p = ttest_ind(group_scores, reference_scores, equal_var=False, alternative='less')
#     return p


def fair_lending_disparity(frame, y, yhat, demo_name, groups, reference_group, cutoff=0.5, favorable_outcome=0):
    """
    Creates a table of fair lending disparity metrics (AIR and SMD).

    Args:
        frame: Pandas dataframe of actual (y) and predicted (yhat) values with group membership.
        y: Name of actual value column.
        yhat: Name of predicted value column.
        demo_name: The name of the column containing the group information
        groups: The names of the groups in the demo_name column.
        reference_group: The control group.
        cutoff: Cutoff threshold for confusion matrix, default 0.5.
        favorable_outcome: The value {0, 1} that corresponds to a desirable outcome.

    Returns:
        A DataFrame summarizing the fair lending metrics analysis
    """

    protected_groups = [group for group in groups if group != reference_group]
    groups_ordered = protected_groups + [reference_group]

    temp_frame = frame.copy()
    temp_frame['model_outcome'] = np.where(temp_frame[yhat] <= cutoff, 0, 1)
    temp_frame['fav_outcome'] = temp_frame['model_outcome'] == favorable_outcome
    temp_frame['fav_score'] = temp_frame[yhat] if favorable_outcome else 1-temp_frame[yhat]

    disparity_table = pd.DataFrame(index=groups_ordered)

    disparity_table['Count'] = [len(temp_frame.loc[temp_frame[demo_name] == group]) for group in groups_ordered]
    disparity_table['Favorable Outcomes'] = [temp_frame.loc[temp_frame[demo_name] == group]['fav_outcome'].sum()
                                             for group in groups_ordered]
    disparity_table['Favorable Rate'] = [disparity_table['Favorable Outcomes'][group]/disparity_table['Count'][group]
                                         for group in groups_ordered]
    disparity_table['Mean Score'] = [temp_frame.loc[temp_frame[demo_name] == group][yhat].mean()
                                             for group in groups_ordered]
    disparity_table['Std Score'] = [temp_frame.loc[temp_frame[demo_name].isin([reference_group, group])][yhat].std()
                                             for group in groups_ordered]
    try:
        disparity_table['AIR'] = [disparity_table['Favorable Rate'][group]/disparity_table['Favorable Rate'][reference_group]
                                  for group in groups_ordered]
    except:
        disparity_table['AIR'] = np.nan

    # disparity_table['AIR p-value'] = [air_statistical_signif(disparity_table['Count'][group],
    #                                                          disparity_table['Favorable Outcomes'][group],
    #                                                          disparity_table['Count'][reference_group],
    #                                                          disparity_table['Favorable Outcomes'][reference_group])
    #                                   for group in groups_ordered]

    disparity_table['SMD'] = [(disparity_table['Mean Score'][group] -
                               disparity_table['Mean Score'][reference_group]) /
                              disparity_table['Std Score'][group]
                              for group in groups_ordered]

    # disparity_table['SMD p-value'] = [smd_statistical_signif(temp_frame.loc[temp_frame[demo_name] == group]['fav_score'],
    #                                                          temp_frame.loc[temp_frame[demo_name] == reference_group]['fav_score'])
    #                                   for group in groups_ordered]

    return disparity_table


In [ ]:
# Calculate the AIR and SMD scores across Train, Test & Validation datasets.

fair_lending_disparity(train, y=target, yhat=f'p_{target}',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=best_cut)

In [ ]:
# @title Excercise : Disparity for validation set
fair_lending_disparity(valid, y=target, yhat=f'p_{target}',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=best_cut)

In [ ]:
# @title Excersise : Disparity for test set

fair_lending_disparity(test, y=target, yhat=f'p_{target}',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=best_cut)

### Residual Fairness Analysis

We can also simply check the residual of the model, to see how does error relate to within each group.

In [ ]:
def plot_group_residuals(frame, y, yhat, demo_name, groups, cutoff=0.5, favorable_outcome=0):
    """
    Generates a plot of logloss residuals for individuals who falsely received an unfavorable outcome.

    Args:
        frame: Pandas dataframe of actual (y) and predicted (yhat) values with group membership.
        y: Name of actual value column.
        yhat: Name of predicted value column.
        demo_name: The name of the column containing the group information
        groups: The names of the groups in the demo_name column.
        cutoff: Cutoff threshold for confusion matrix, default 0.5.
        favorable_outcome: The value {0, 1} that corresponds to a desirable outcome.

    Returns:
        Axes of the plot
    """

    fig, axs = plt.subplots(1, len(groups), figsize=(5*len(groups), 5))
    for i, group in enumerate(groups):
        group_frame = frame.loc[frame[demo_name] == group].copy()
        group_frame[f'{yhat}_outcome'] = np.where(group_frame[yhat] > cutoff, 1, 0)

        group_frame[f'{y}_residual'] = -group_frame[y]*np.log(group_frame[yhat])
        group_frame[f'{y}_residual'] -= (1 - group_frame[y])*np.log(1 - group_frame[yhat])

        misclassified_obs = group_frame[(group_frame[y] == favorable_outcome) &
                                        (group_frame[f'{yhat}_outcome'] == (1 - favorable_outcome))]
        axs[i].scatter(misclassified_obs[yhat], misclassified_obs[f'{y}_residual'], marker='o', alpha=0.3)
        axs[i].set_xlabel('Score')
        axs[i].set_ylabel('Residual')
        axs[i].set_title(group)
        axs[i].set_xlim([0, 1])
        axs[i].set_ylim([0, 2])

    fig.suptitle('Logloss Residuals')

    return axs

In [ ]:
_ = plot_group_residuals(test, y=target, yhat=f'p_{target}',
                       demo_name='RACE', groups=race_levels,
                       cutoff=best_cut)


In [ ]:
# @title Excersie : Residual Plot for Gender

gender_levels = [] # define gender level as list ['male','female']

if len(gender_levels) ==0 :
  print("Define gender levels")
else :
  _ = plot_group_residuals(test, y=target, yhat=f'p_{target}',
                       demo_name='SEX', groups=gender_levels,
                       cutoff=best_cut)

## 3. Remediating Model Bias


<img src="https://raw.githubusercontent.com/anilkumarpanda/fairness_cb/development/img/ds_cat.JPG" alt="Alt Text" height="400">

A cat data scientist at work.

There are many potential ways to remediate bias. Some of the most common include pre-, in-, and postprocessing, and model selection:

1. Preprocessing : Rebalancing, reweighing, or resampling training data so that demographic groups are better represented or positive outcomes are distributed more equitably.

2. In-processing : Any number of alterations to ML training algorithms, including constraints, regularization and dual loss functions, or incorporation of adversarial modeling information, that attempt to generate more balanced outputs or performance across demographic groups.

3. Postprocessing :Changing model predictions directly to create less biased outcomes.

4. Model selection : Considering bias along with performance when selecting models. Typically, it’s possible to find a model with good performance and fairness characteristics if we measure bias and performance across a large set of hyperparameter settings and input features.

### Pre-Processing

In [ ]:
def reweight_dataset(frame, y, demo_name, groups):
    """
    Generates a weight for each observation according to the reweighting algorithm of
    Kamiran and Kalders 2012, Data preprocessing techniques for classification without discrimination.

    Args:
        frame: Pandas dataframe of actual (y) and group information.
        y: Name of actual value column (assumed to be binary).
        demo_name: The name of the column containing the group information
        groups: The names of the groups in the demo_name column.

    Returns:
        A Series containing the new observation weights.
    """

    n = len(frame)

    freq_dict = {'pos': len(frame.loc[frame[y] == 1])/n,
                 'neg': len(frame.loc[frame[y] == 0])/n}

    freq_dict.update({group: frame[demo_name].value_counts()[group]/n for group in groups})

    weights = pd.Series(np.ones(n), index=frame.index)

    for label in [0, 1]:
        for group in groups:
            label_name = 'pos' if label == 1 else 'neg'
            freq = frame.loc[frame[y] == label][demo_name].value_counts()[group]/n
            weights[(frame[y] == label) & (frame[demo_name] == group)] *= freq_dict[group]*freq_dict[label_name]/freq

    return weights

In [ ]:
train_weights = reweight_dataset(train, target, 'RACE', race_levels)

In [ ]:
for race in race_levels:
    print(f"Mean outcome for {race}: {np.round(train.loc[train['RACE'] == race][target].mean(), 3)}")

    weighted_target = np.multiply(train.loc[train['RACE'] == race][target],
                                  train_weights.loc[train['RACE'] == race])

    print(f"Mean outcome for {race} - reweighted: {np.round(weighted_target.mean(), 3)} \n")

In [ ]:
dtrain = xgb.DMatrix(train[features],
                     label=train[target],
                     weight=train_weights)


model_reweighted = xgb.train(params,
                             dtrain,
                             num_boost_round=100,
                             evals=watchlist,
                             early_stopping_rounds=10,
                             verbose_eval=False)

In [ ]:
reweighted_model_metrics = perf_metrics(y_true=valid[target], y_score=model_reweighted.predict(dvalid))
reweighted_best_cut = reweighted_model_metrics.loc[reweighted_model_metrics['f1'].idxmax(), 'cutoff']

In [ ]:
test[f'p_{target}_reweighted'] = model_reweighted.predict(xgb.DMatrix(test[features], label=test[target]))

fair_lending_disparity(test, y=target, yhat=f'p_{target}_reweighted',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=reweighted_best_cut)

In [ ]:
train[f'p_{target}_reweighted'] = model_reweighted.predict(dtrain)

fair_lending_disparity(train, y=target, yhat=f'p_{target}_reweighted',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=reweighted_best_cut)

In [ ]:
def make_fair_objective(protected, lam):
    def fair_objective(pred, dtrain):
        """
        Fairness-aware cross-entropy loss objective function
        """
        label = dtrain.get_label()
        pred = 1.0 / (1.0 + np.exp(-pred))
        grad = (pred - label) - lam * (pred - protected)
        hess = (1 - lam) * pred * (1. - pred)

        return grad, hess
    return fair_objective

protected = np.where((train['RACE'] == 'hispanic') | (train['RACE'] == 'black'), 1, 0)
fair_objective = make_fair_objective(protected, lam=0.2)


In [ ]:
dtrain = xgb.DMatrix(train[features],
                     label=train[target])


# Train using early stopping on the validation dataset.
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

model_regularized = xgb.train(params,
                              dtrain,
                              num_boost_round=100,
                              evals=watchlist,
                              early_stopping_rounds=10,
                              verbose_eval=False,
                              obj=fair_objective)

In [ ]:
model_metrics = perf_metrics(y_true=valid[target], y_score=model_regularized.predict(dvalid))
best_cut_regularized = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

test[f'p_{target}_regularized'] = model_regularized.predict(xgb.DMatrix(test[features], label=test[target]))

fair_lending_disparity(test, y=target, yhat=f'p_{target}_regularized',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=best_cut_regularized)

In [ ]:
train[f'p_{target}_regularized'] = model_regularized.predict(xgb.DMatrix(train[features], label=train[target]))

fair_lending_disparity(train, y=target, yhat=f'p_{target}_regularized',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=best_cut_regularized)

In [ ]:
# Generate plots of model accuracy (F1 score), number of trees,
# black AIR, hispanic AIR, and asian AIR as a
# function of the regularization hyperparameter.

lams = np.arange(0, 1, 0.10)
scores = []
trees = []

air_black = []
air_hispanic = []
air_asian = []

dtest = xgb.DMatrix(test[features], label=test[target])

for lam in lams:

    model = xgb.train(params,
                      dtrain,
                      num_boost_round=200,
                      evals=watchlist,
                      early_stopping_rounds=10,
                      verbose_eval=False,
                      obj=make_fair_objective(protected, lam=lam))

    model_metrics = perf_metrics(y_true=valid[target],
                                 y_score=model.predict(dvalid, iteration_range=(0, model.best_iteration)))
    best_cut = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

    test[f'p_{target}_lam_test'] = model.predict(dtest, iteration_range=(0, model.best_iteration))

    air_table = fair_lending_disparity(test, y=target, yhat=f'p_{target}_lam_test',
                                       demo_name='RACE', groups=race_levels, reference_group='white',
                                       cutoff=best_cut)

    scores.append(model_metrics['f1'].max())
    trees.append(model.best_ntree_limit)

    air_black.append(air_table.loc['black']['AIR'])
    air_hispanic.append(air_table.loc['hispanic']['AIR'])
    air_asian.append(air_table.loc['asian']['AIR'])


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].plot(lams, [score/scores[0] for score in scores])
axs[0].set_xlabel('Lambda')
axs[0].set_ylabel(f'F1 Score \n Relative to original')

axs[1].plot(lams, trees)
axs[1].set_xlabel('Lambda')
axs[1].set_ylabel('Num Trees')

#fig.savefig('../Data/Data/Figures/inprocessing_summary_i.svg')

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
axs[0].plot(lams, air_black)
axs[0].set_xlabel('Lambda')
axs[0].set_ylabel('AIR (black)')
axs[0].set_ylim((air_black[0]-0.05, 1.05))

axs[1].plot(lams, air_hispanic)
axs[1].set_xlabel('Lambda')
axs[1].set_ylabel('AIR (Hispanic)')
axs[1].set_ylim((air_hispanic[0]-0.05, 1.05))


axs[2].plot(lams, air_asian)
axs[2].set_xlabel('Lambda')
axs[2].set_ylabel('AIR (asian)')
axs[2].set_ylim((air_hispanic[0]-0.05, 1.05))

#fig.savefig('../Data/Data/Figures/inprocessing_summary_ii.png')

### Excercise
1. How does re-weighting the classes affect AIR ?
2. Can you think of possible scenarios where re-weighting the data would help ?

In [ ]:
# @title Excercise : Reweighting Scheme and its effect on AIR

# Conduct a similar experiment with the magnitude of the reweighing scheme.

magnitudes = np.arange(0, 1.5, 0.10)
scores = []
trees = []

air_black_ii = []
air_hispanic_ii = []
air_asian_ii = []

dtest = xgb.DMatrix(test[features], label=test[target])

for mag in magnitudes:

    weight_change = 1 - train_weights
    new_train_weights = 1 - mag*weight_change

    dtrain = xgb.DMatrix(train[features],
                     label=train[target],
                     weight=new_train_weights)


    model = xgb.train(params,
                      dtrain,
                      num_boost_round=200,
                      evals=watchlist,
                      early_stopping_rounds=10,
                      verbose_eval=False)

    model_metrics = perf_metrics(y_true=valid[target],
                                 y_score=model.predict(dvalid, iteration_range=(0, model.best_iteration)))
    best_cut = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

    test[f'p_{target}_lam_test'] = model.predict(dtest, iteration_range=(0, model.best_iteration))

    air_table = fair_lending_disparity(test, y=target, yhat=f'p_{target}_lam_test',
                                       demo_name='RACE', groups=race_levels, reference_group='white',
                                       cutoff=best_cut)

    scores.append(model_metrics['f1'].max())
    trees.append(model.best_ntree_limit)

    air_black_ii.append(air_table.loc['black']['AIR'])
    air_hispanic_ii.append(air_table.loc['hispanic']['AIR'])
    air_asian_ii.append(air_table.loc['asian']['AIR'])


In [ ]:
# @title Excersise : Reweighting effect on model performance

# This plot shows a relative drop in F1 score down to 94% of the original value.
# Meanwhile, black and Hispanic AIRs go from 0.72 to 0.9!

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].plot(magnitudes, [score/scores[0] for score in scores])
axs[0].set_xlabel('Reweighing Strength')
axs[0].set_ylabel('F1 Score')

axs[1].plot(magnitudes, trees)
axs[1].set_xlabel('Reweighing Strength')
axs[1].set_ylabel('Num Trees')

#fig.savefig('../Data/Data/Figures/preprocessing_summary_i.svg')

In [ ]:
# @title Excersise : Reweighting effect on AIR across Race groups

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].plot(magnitudes, air_black_ii)
axs[0].set_xlabel('Reweighing Strength')
axs[0].set_ylabel('AIR (black)')
axs[0].set_ylim((air_black_ii[0]-0.05, 1.05))

axs[1].plot(magnitudes, air_hispanic_ii)
axs[1].set_xlabel('Reweighing Strength')
axs[1].set_ylabel('AIR (Hispanic)')
axs[1].set_ylim((air_hispanic_ii[0]-0.05, 1.05))

axs[2].plot(magnitudes, air_asian_ii)
axs[2].set_xlabel('Reweighing Strength')
axs[2].set_ylabel('AIR (asian)')
axs[2].set_ylim((air_hispanic[0]-0.05, 1.05))

#fig.savefig('../Data/Data/Figures/preprocessing_summary_ii.png')

## 4. Model Selection

### Feature Selection

Feature selection is a powerful technique for debiasing. We'll look at the effect of dropping each feature in turn.

We are reaching the end , Stay Strong !!

<img src="https://raw.githubusercontent.com/anilkumarpanda/fairness_cb/development/img/red_cat.JPG" alt="Alt Text" height="400">

### Excercise
1. This code takes a while a run, in the meanwhile can you come up with an
easier way to identify features that can be adding addtional bias to the model ?

Hint : Adversarial Models ?

In [ ]:
# We'll examine the effect of dropping each feature individually on model performance and adverse impact ratios.
features_to_drop = ['original model'] + features

In [ ]:
num_cv_folds = 5

# Build the custom cross-validation iterable.
all_indices = np.arange(0, len(train))
all_indices = np.random.permutation(all_indices)
splits = np.array([int(np.floor(len(train)/num_cv_folds)) for _ in range(num_cv_folds-1)])
splits = np.append(splits, len(train) - splits.sum())

test_indices = np.split(all_indices, splits.cumsum())[:-1]
test_groups = [train.iloc[test_ind]['RACE'].values for test_ind in test_indices]

train_indices = [np.array([i for i in all_indices if i not in test_ind]) for test_ind in test_indices]

In [ ]:
feature_selection_results = pd.DataFrame(index=features_to_drop,
                                         columns=['AUC', 'Black AIR', 'Hispanic AIR'])

for dropped_feature in features_to_drop:

    new_features = list(set(features).difference(set([dropped_feature])))

    cv_auc = []
    cv_black_air = []
    cv_hispanic_air = []

    for fold_num, (train_ind, test_ind) in enumerate(zip(train_indices, test_indices)):

        new_monotone_constraints={k: v for k, v in dict(zip(features, monotone_constraints)).items() if k != dropped_feature}

        cv_model = xgb.XGBClassifier(n_estimators=150,
                                     max_depth=5,
                                     learning_rate=0.05,
                                     subsample=0.6,
                                     colsample_bytree=1.0,
                                     monotone_constraints=new_monotone_constraints,
                                     random_state=12345,
                                     use_label_encoder=False,
                                     base_score=params['base_score'],
                                     eval_metric='logloss',n_jobs=8)

        train_slice = train.reset_index(drop=True).iloc[train_ind].copy()
        test_slice = train.reset_index(drop=True).iloc[test_ind].copy()

        cv_model = cv_model.fit(train_slice[new_features],
                                train_slice[target])
        y_pred = cv_model.predict_proba(test_slice[new_features])[:, 1]


        model_metrics = perf_metrics(y_true=train[target].values[test_ind], y_score=y_pred)
        best_cut = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

        cv_auc.append(sklearn.metrics.roc_auc_score(y_true=train[target].values[test_ind],
                                                    y_score=y_pred))

        test_slice['pred'] = y_pred
        disparity_table = fair_lending_disparity(test_slice, y=target, yhat=f'pred',
                           demo_name='RACE', groups=race_levels, reference_group='white',
                           cutoff=best_cut)

        cv_black_air.append(disparity_table.loc['black']['AIR'])
        cv_hispanic_air.append(disparity_table.loc['hispanic']['AIR'])

    feature_selection_results.loc[dropped_feature] = [np.mean(cv_auc),
                                                      np.mean(cv_black_air),
                                                      np.mean(cv_hispanic_air)]

In [ ]:
feature_to_drop = feature_selection_results.sort_values('Black AIR').index[-1]
feature_to_drop

In [ ]:
feature_selection_results.sort_values('Black AIR')

In [ ]:
# Drop the most 'problematic feature' calculate model scores.
new_features = list(set(features).difference(set([feature_to_drop])))
new_monotone_constraints={k: v for k, v in dict(zip(features, monotone_constraints)).items() if k in new_features}

### Hyperparameter Tuning

In [ ]:
parameter_distributions = {
    'n_estimators': np.arange(10, 221, 30),
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.7, 0.3),
    'colsample_bytree': stats.uniform(0.5, 0.5),
    'reg_lambda': stats.uniform(0.1, 50),
    'monotone_constraints': [new_monotone_constraints],
    'base_score': [params['base_score']]
    }

In [ ]:
fold_number = -1

def black_air(y_true, y_pred):


    global fold_number
    fold_number = (fold_number + 1) % num_cv_folds

    model_metrics = perf_metrics(y_true, y_score=y_pred)
    best_cut = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

    data = pd.DataFrame({'RACE': test_groups[fold_number],
                        'y_true': y_true,
                        'y_pred': y_pred},
                        index=np.arange(len(y_pred)))

    disparity_table = fair_lending_disparity(data, y='y_true', yhat='y_pred',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=best_cut)

    return disparity_table.loc['black']['AIR']


def hispanic_air(y_true, y_pred):


    global fold_number

    model_metrics = perf_metrics(y_true, y_score=y_pred)
    best_cut = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

    data = pd.DataFrame({'RACE': test_groups[fold_number],
                        'y_true': y_true,
                        'y_pred': y_pred},
                        index=np.arange(len(y_pred)))

    disparity_table = fair_lending_disparity(data, y='y_true', yhat='y_pred',
                       demo_name='RACE', groups=race_levels, reference_group='white',
                       cutoff=best_cut)

    return disparity_table.loc['hispanic']['AIR']

scoring = {
        'AUC': 'roc_auc',
        'Black AIR': sklearn.metrics.make_scorer(black_air, needs_proba=True),
        'Hispanic AIR': sklearn.metrics.make_scorer(hispanic_air, needs_proba=True)
    }

In [ ]:
# @title Excersise : Increase the no.of iterations.
# Increase the no.of iterations to 50.


In [ ]:
grid_search = sklearn.model_selection.RandomizedSearchCV(xgb.XGBClassifier(random_state=12345,
                                                                           use_label_encoder=False,
                                                                           eval_metric='logloss'),
                                                         parameter_distributions,
                                                         n_iter=10,
                                                         scoring=scoring,
                                                         cv=zip(train_indices, test_indices),
                                                         refit=False,
                                                         error_score='raise').fit(train[new_features], train[target].values)
results = pd.DataFrame(grid_search.cv_results_)

In [ ]:
original_auc = feature_selection_results.loc['original model']['AUC']
original_black_air = feature_selection_results.loc['original model']['Black AIR']

new_auc = feature_selection_results.loc[feature_to_drop]['AUC']
new_black_air = feature_selection_results.loc[feature_to_drop]['Black AIR']

fig, ax = plt.subplots()
ax.scatter(original_black_air, 1.0, s=50, color='red', label='original model')
ax.scatter(results['mean_test_Black AIR'], results['mean_test_AUC']/original_auc, label='post-hyperparameter tuning', marker='x')
ax.scatter(new_black_air, new_auc/original_auc, color='orange', label='post-feature selection', marker='v')
ax.legend(loc='lower left')
ax.set_xlabel('Black AIR')
ax.set_ylabel(f'AUC (normalized)')

#fig.savefig('../Data/Data/Figures/model_tuning_scatter.svg', dpi=300)

In [ ]:
results[['mean_test_Black AIR', 'mean_test_Hispanic AIR', 'mean_test_AUC']].sort_values(['mean_test_Black AIR', 'mean_test_AUC'], ascending=False)
highest_air_idx = results.loc[results['mean_test_Black AIR'] == results['mean_test_Black AIR'].max()].index[0]

# We can also choose the fairest model that demonstrates no more than a 1% decrease in AUC from the original model.
business_viable_models = results.loc[results['mean_test_AUC'] >= 0.99*original_auc]
alternative_model_idx = business_viable_models.loc[business_viable_models['mean_test_Black AIR'] == business_viable_models['mean_test_Black AIR'].max()].index[0]


In [ ]:
new_hyperparameter_idx = [highest_air_idx, alternative_model_idx]

tuned_params = ['_'.join(col.split('_')[1:]) for col in results.columns if col.startswith('param_')]
new_hyperparameters = [dict(zip(tuned_params, [row[f'param_{param}'] for param in tuned_params])) for _, row in results.loc[new_hyperparameter_idx].iterrows()]

### Conclusion

In [ ]:
def model_summary(y_true, y_pred, group_info, reference_group, metric_dict, cutoff,
                  confusion_metrics_to_show=['False Positive Rate']):
    """
    Function to put all things together.

    """

    groups = np.unique(group_info)
    protected_groups = [group for group in groups if group != reference_group]

    model_metrics = perf_metrics(y_true, y_score=y_pred)
    # best_cut = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']
    # f1 = model_metrics['f1'].max()

    f1 = model_metrics.loc[model_metrics['cutoff'] == cutoff, 'f1'].values[0]

    auc = sklearn.metrics.roc_auc_score(y_true, y_score=y_pred)

    data = pd.DataFrame({'demo': group_info,
                        'y_true': y_true,
                        'y_pred': y_pred},
                        index=np.arange(len(y_pred)))

    disparity_table = fair_lending_disparity(data, y='y_true', yhat='y_pred',
                       demo_name='demo', groups=groups, reference_group=reference_group,
                       cutoff=cutoff)

    airs = dict(zip([f'{group} AIR' for group in protected_groups],
                    [disparity_table.loc[group]['AIR'] for group in protected_groups]))

    confusion_mats = {level: get_confusion_matrix(data, 'y_true', 'y_pred', by='demo',
                                                  level=level, cutoff=cutoff) for level in groups}
    confusion_metrics = confusion_matrix_metrics(confusion_mats, metric_dict)
    confusion_disparity_frame = confusion_metrics/confusion_metrics.loc[reference_group, :]

    confusion_metric_disparities = dict()
    for metric in confusion_metrics_to_show:
        confusion_metric_disparities.update(dict(zip([f"{group} {metric} Disparity" for group in protected_groups],
                                               [confusion_disparity_frame.loc[group][metric] for group in protected_groups])))

    output = {'AUC': auc, 'F1': f1}
    output.update(airs)
    output.update(confusion_metric_disparities)

    return pd.Series(output)

In [ ]:

feature_selection_model = xgb.XGBClassifier(n_estimators=150,
                                     max_depth=5,
                                     learning_rate=0.05,
                                     subsample=0.6,
                                     colsample_bytree=1.0,
                                     monotone_constraints=new_monotone_constraints,
                                     random_state=12345,
                                     use_label_encoder=False,
                                     base_score=params['base_score'],
                                     eval_metric='logloss').fit(train[new_features], train[target])

model_metrics = perf_metrics(y_true=valid[target],
                             y_score=feature_selection_model.predict_proba(valid[new_features])[:, 1])
best_cut_feature_selection = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

hyp_tuning_model_1 = xgb.XGBClassifier(random_state=12345,
                                     use_label_encoder=False,
                                     eval_metric='logloss', **new_hyperparameters[0]).fit(train[new_features], train[target])

model_metrics = perf_metrics(y_true=valid[target],
                             y_score=hyp_tuning_model_1.predict_proba(valid[new_features])[:, 1])
best_cut_hyp_1 = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']

hyp_tuning_model_2 = xgb.XGBClassifier(random_state=12345,
                                     use_label_encoder=False,
                                     eval_metric='logloss', **new_hyperparameters[1]).fit(train[new_features], train[target])


model_metrics = perf_metrics(y_true=valid[target],
                             y_score=hyp_tuning_model_2.predict_proba(valid[new_features])[:, 1])
best_cut_hyp_2 = model_metrics.loc[model_metrics['f1'].idxmax(), 'cutoff']


In [ ]:
model_predictions = {'Original Model': (test[f"p_{target}"].values, best_cut_original),
                    'Pre-processing (reweighting)': (test[f'p_{target}_reweighted'].values, reweighted_best_cut),
                    'Feature selection': (feature_selection_model.predict_proba(test[new_features])[:, 1], best_cut_feature_selection),
                    'HPT :1 Fairest Model': (hyp_tuning_model_1.predict_proba(test[new_features])[:, 1], best_cut_hyp_1),
                    'HPT :2 Balance Fairness & Performance': (hyp_tuning_model_2.predict_proba(test[new_features])[:, 1], best_cut_hyp_2)}

In [ ]:
model_summaries = {name: model_summary(test[target].values, pred, test['RACE'].values, 'white', metric_dict, cutoff) for name, (pred, cutoff) in model_predictions.items()}

### Time to make a choice :
Put your money on the models!

<img src="https://raw.githubusercontent.com/anilkumarpanda/fairness_cb/development/img/banker_cat.JPG" alt="Alt Text" height="400">

In [ ]:
pd.DataFrame(model_summaries)



### References :

Most of the code and knowledge is borrowed from the following sources :

1. [Fairness Tree](https://docs.google.com/presentation/d/1ycNhDJr5uQiBhWvdlNArnv7ojydW73qryOjRBB30Z44/edit#slide=id.g8d5290cc44_0_417)
2. [Machine Learning for High-Risk Applications](https://learning.oreilly.com/library/view/machine-learning-for/9781098102425/)
3. [SHAP for Fairness](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/Explaining%20quantitative%20measures%20of%20fairness.html)